In [2]:
from jupyter_dash import JupyterDash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
import plotly.express as px
import base64
import os
from CRUD_Python_Module import AnimalShelter

JupyterDash.infer_jupyter_proxy_config()

db = AnimalShelter()
data = db.read({})
df = pd.DataFrame.from_records(data)
if '_id' in df.columns:
    df = df.drop('_id', axis=1)

app = JupyterDash(__name__)

logo_file = 'Grazioso Salvare Logo.png'
if os.path.exists(logo_file):
    encoded_logo_file = base64.b64encode(open(logo_file, 'rb').read())
    logo = html.Img(src='data:image/png;base64,{}'.format(encoded_logo_file.decode()), style={'height':'100px'})
else:
    logo = html.Div("Logo not found!")

app.layout = html.Div([
    html.Div([logo, html.Div("Dashboard by Justin Perez")], style={'textAlign':'center'}),
    html.Hr(),
    html.Label("Choose Rescue Type:"),
    dcc.RadioItems(
        id='rescue-type',
        options=[
            {'label':'Water Rescue', 'value':'Water Rescue'},
            {'label':'Mountain or Wilderness Rescue', 'value':'Mountain or Wilderness Rescue'},
            {'label':'Disaster or Individual Tracking', 'value':'Disaster or Individual Tracking'},
            {'label':'Reset', 'value':'Reset'}
        ],
        value='Reset'
    ),
    html.Hr(),
    dash_table.DataTable(
        id='dog-data-table',
        columns=[{'name': c, 'id': c} for c in df.columns],
        data=df.to_dict('records'),
        page_size=5,
        row_selectable='single'
    ),
    html.Hr(),
    html.Div(id='graph'),
    html.Div(id='map')
])

@app.callback(Output('dog-data-table', 'data'),
              [Input('rescue-type', 'value')])
def filter_table(rescue_type):
    if rescue_type == 'Reset':
        filtered = db.read({})
    else:
        breeds = []
        age_limit = 104
        if rescue_type == 'Water Rescue':
            breeds = ['Labrador Retriever', 'Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland', 'Portuguese Water Dog', 'Flat-Coated Retriever', 'Golden Retriever', 'Golden Retriever Mix']
        if rescue_type == 'Mountain or Wilderness Rescue':
            breeds = ['German Shepherd', 'Border Collie', 'Belgian Malinois', 'Australian Shepherd', 'Bloodhound', 'Collie', 'Shetland Sheepdog', 'Husky']
        if rescue_type == 'Disaster or Individual Tracking':
            breeds = ['Bloodhound', 'German Shepherd', 'Belgian Malinois', 'Doberman Pinscher', 'Labrador Retriever', 'Beagle', 'Golden Retriever']
        query = {'animal_type':'Dog','age_upon_outcome_in_weeks':{'$lte':age_limit}}
        if breeds:
            query['breed'] = {'$in':breeds}
        filtered = db.read(query)
    df_filtered = pd.DataFrame.from_records(filtered)
    if '_id' in df_filtered.columns:
        df_filtered = df_filtered.drop('_id', axis=1)
    return df_filtered.to_dict('records')

@app.callback(Output('graph','children'),
              [Input('dog-data-table','derived_virtual_data')])
def update_pie(data_table):
    if data_table is None:
        df2 = df.copy()
    else:
        df2 = pd.DataFrame.from_dict(data_table)
    if df2.empty or 'breed' not in df2.columns:
        return [html.Div("No data available")]
    df2 = df2[df2['breed'].notnull()]
    df2['breed'] = df2['breed'].astype(str)
    if df2.empty:
        return [html.Div("No breed info for this filter")]
    fig = px.pie(df2, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

@app.callback(Output('map','children'),
              [Input('dog-data-table','derived_virtual_data'),
               Input('dog-data-table','selected_rows')])
def update_map(data_table, selected_rows):
    if data_table is None:
        return
    df2 = pd.DataFrame.from_dict(data_table)
    if df2.empty:
        return html.Div("No data to map")
    if selected_rows is None or len(selected_rows) == 0:
        row = 0
    else:
        row = selected_rows[0]
    lat, lon = None, None
    for c in ['location_lat','latitude','lat']:
        if c in df2.columns:
            lat = df2.iloc[row][c]
            break
    for c in ['location_long','longitude','lon']:
        if c in df2.columns:
            lon = df2.iloc[row][c]
            break
    if lat is None or lon is None:
        return html.Div("No coordinates")
    breed = df2.iloc[row]['breed'] if 'breed' in df2.columns else ''
    name = df2.iloc[row]['name'] if 'name' in df2.columns else ''
    return [dl.Map(center=[lat, lon], zoom=10, style={'width':'100%','height':'400px'},
                   children=[dl.TileLayer(),
                             dl.Marker(position=[lat, lon],
                                       children=[dl.Tooltip(breed),
                                                 dl.Popup([html.H4("Name"), html.P(name)])])])]

app.run_server(mode='inline', debug=False)




 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Oct/2025 18:39:39] "GET /_alive_9c05de75-2379-4722-b747-06d4c47b517a HTTP/1.1" 200 -


127.0.0.1 - - [13/Oct/2025 18:39:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:40] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:40] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:40] "GET /_dash-component-suites/dash/dash_table/async-highlight.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Oct/2025 18:39:40] "GET /_dash-component-suites/dash/dash_table/async-table.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Oct/2025 18:39:40] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:41] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2025 18:39:42] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [13/Oct/2025 18:39:42] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - -